In [ ]:
{-# LANGUAGE OverloadedStrings #-}
import Control.Lens
import Network.Wreq
import Data.String
import qualified Data.Text as T
import Data.Aeson.Lens
import Data.HashMap.Strict as HP

In [ ]:
options = defaults 
            & param "jql" .~ ["issuekey = REST-248 OR issuekey = SANDBOX-49"]
            & param "maxResults" .~ [fromString $ show 5]

issues <- getWith options "https://ecosystem.atlassian.net/rest/api/2/search"

In [ ]:
issueIds issues =
        issues ^. responseBody 
                 . key "issues" 
                 . _Array 
                 . traverse 
                 . key "key"
                 . _String
createJQL issueIds = T.intercalate " OR " $
                     IHaskellPrelude.map ("issuekey = " `T.append`) issueIds 

In [ ]:
issuesArray = (issues ^. responseBody 
                        . key "issues" 
                        . _Array)
(over each (\issue -> (issue ^. key "key" ._String,
                       issue ^.. key "fields" 
                               . key "subtasks" 
                               . _Array . traverse 
                               . key "key"
                               . _String )) 
      issuesArray)

In [ ]:
(issues ^.. responseBody 
          . key "issues" 
          . _Array 
          . traverse 
          . key "fields"
          )